In [16]:
from dag_cld import env
from dag_cld import ast
from dag_cld import mask
from dag_cld import teacher

In [17]:
logger = env.Logger(blabla=False)
fop = env.File(logger)

pmask = mask.Polar(logger)

fts = ast.Fits(logger)

ima = ast.Image(logger)

svm = teacher.SVM(logger)
cnn = teacher.CNN(logger)
knn = teacher.KNN(logger)
lr = teacher.LR(logger)
nb = teacher.NB(logger)

In [18]:
number_of_samples = 6
mask_coordinates = {"E": ((20, 70), (45, 135)), "S": ((20, 70), (135, 225)),
                    "W": ((20, 70), (225, 315)), "N": ((20, 70), (315, 405)),
                    "ZE": ((70, 90), (45, 135)), "ZS": ((70, 90), (135, 225)),
                    "ZW": ((70, 90), (225, 315)), "ZN": ((70, 90), (315, 405))}

In [19]:
for direction, coordinates in mask_coordinates.items():
    sky_files = fop.list_in_path("data/clear/*_S_vec.fits.gz".format(direction))
    cld_files = fop.list_in_path("data/cloud/*_S_vec.fits.gz".format(direction))
    
    random_sky_files = svm.random_choices(sky_files, number_of_samples)
    random_cld_files = svm.random_choices(cld_files, number_of_samples)
    
    random_sky_vectors = []
    for random_sky_file in random_sky_files:
        vec = fts.data(random_sky_file)
        random_sky_vectors.append(vec)

    random_sky_vectors = ima.list2array(random_sky_vectors)
        
    random_cld_vectors = []
    for random_cld_file in random_cld_files:
        vec = fts.data(random_cld_file)
        random_cld_vectors.append(vec)

    random_cld_vectors = ima.list2array(random_cld_vectors)
    whole_classes = svm.class_combiner(
    svm.class_adder(random_sky_vectors, 1),
    svm.class_adder(random_cld_vectors, 0))

    whole_classes = svm.shuffle(whole_classes)

    training, test = svm.tts(whole_classes, test_size=0.20)

    X_train, y_train = training[:, :-1], training[:, -1]
    X_test, y_test = test[:, :-1], test[:, -1]

    clsf = svm.classifier(X_train, y_train)

    svm.save(clsf, "clsf/svm_day_{}.clsf".format(direction))